# Object-oriented programming
In the previous notebook, we implemented a simple class for the Portfolio class that allows the programmer to keep track of the positions in their portfolio. In this notebook, we will further extend this portfolio class to allow some more complex behavior. Three different advanced concepts will be introduced:
1. Dunder methods (also called magic methods)
2. Different ways of defining class attributes
3. Class- and static methods

In [1]:
from Portfolio import Portfolio
            
positions = {"Apple":   {"Share price": 150.0,  "Number": 1000},
             "Verizon": {"Share price": 52.4,   "Number": 2000},
             "Solvay":  {"Share price": 106.3,  "Number": 2000}}
initial_cash = 1e6
portfolio = Portfolio(initial_cash=initial_cash, positions=positions)

print(portfolio)

A portfolio consisting of 3 positions and a total value of 1,467,400.0 USD.


## Dunder methods
Two dunder methods have already been specified in the class i.e. the \_\_init\_\_ and \_\_str\_\_ methods. Now, we would like to extend the functionality of the class by specifying additional dunder methods.

### Getting the length of the class
In python, the len() function is often used to get the number of elements in a certain object. For example, len(pd.DataFrame) returns the number of rows in the dataframe and len(list) returns the number of objects in a list. We would like to specify this len(Portfolio) function to return the number of positions in the portfolio.

In [2]:
# Currently
len(portfolio)  # Returns ERROR

3

In [3]:
# After implementing the __len__ dunder method in the portfolio file
len(portfolio) # Returns 3

3

### Defining how to add two portfolios to each other
Sometimes, it could be useful to add the positions of two portfolios up into a new portfolio. Currently, this kind of behavior is not supported.

In [4]:
positions = {"Apple":   {"Share price": 150.0,  "Number": 1000}, # Known position
             "IBM": {"Share price": 52.4,   "Number": 2000}}     # New position
portfolio_2 = Portfolio(initial_cash=2e6, positions=positions)
# Currently
merged_portfolio = portfolio + portfolio_2 # ERROR

In [5]:
print(merged_portfolio)

A portfolio consisting of 4 positions and a total value of 3,722,200.0 USD.


In [6]:
merged_portfolio.positions

{'Apple': {'Share price': 150.0, 'Number': 2000},
 'Verizon': {'Share price': 52.4, 'Number': 2000},
 'Solvay': {'Share price': 106.3, 'Number': 2000},
 'IBM': {'Share price': 52.4, 'Number': 2000}}

### Comparing portfolios with each other
When working with multiple portfolios, we might also be interested in comparing portfolios with each other in terms of size.

In [7]:
# Currently
portfolio >= portfolio_2 # Error

False

### Iterating over the positions in a portfolio
Let's say we would like to see the total exposure for each of the positions in the portfolio. Currently, we need to do this like this:

In [8]:
for company, position in portfolio.positions.items():
    print("Exposure {c}: {exposure} USD".format(
            c=company, 
            exposure=position['Share price']*position['Number']))

Exposure Apple: 300000.0 USD
Exposure Verizon: 104800.0 USD
Exposure Solvay: 212600.0 USD
Exposure IBM: 104800.0 USD


The syntax can be simplified slightly if we implement the \_\_next\_\_ and  \_\_iter\_\_ dunder methods.

In [9]:
for company, position in portfolio:
    print("Exposure {c}: {exposure} USD".format(
            c=company, 
            exposure=position['Share price']*position['Number']))

Exposure Apple: 300000.0 USD
Exposure Verizon: 104800.0 USD
Exposure Solvay: 212600.0 USD
Exposure IBM: 104800.0 USD


## Class attributes
Currently, all attributes are implemented using the classic syntax. However, this creates a couple of very important problems that need to be solved!

1. **The total value of a portfolio is actually an attribute, however, it is only implemented using the get_total_value method** 

In [10]:
portfolio.value # AttributeError

1467400.0

In [11]:
# Note that we cannot modify the value of the portfolio -> which makes sense
#portfolio.value = 1e9

2. **Currently, a user can update the number of shares in a certain position without using the make_transactions method**

In [12]:
portfolio.__dict__.keys()

dict_keys(['_Portfolio__initial_cash', '_Portfolio__positions', 'cash', 'n', 'companies'])

In [13]:
print(f"Pre-transaction cash: {portfolio.cash:,} USD")
print(portfolio.positions["Apple"])
portfolio.positions["Apple"]["Number"] += 100
print(portfolio.positions["Apple"])
print(f"Post-transaction cash: {portfolio.cash:,} USD")

# !! CASH POSITION DID NOT UPDATE AND THIS RETURNED NO ERRORS

Pre-transaction cash: 745,200.0 USD
{'Share price': 150.0, 'Number': 2000}
{'Share price': 150.0, 'Number': 2000}
Post-transaction cash: 745,200.0 USD


Solution? Make the portfolio positions a private attribute! We can then still allow people to make changes but this time only on the copy so it does not affect the actual positions of our portfolio. Then, we can allow the user to explicitly change the positions in the portfolio.

In [14]:
print(f"Pre-transaction cash: {portfolio.cash:,} USD")
print(portfolio.positions["Apple"])
new_positions = portfolio.positions
new_positions["Apple"]["Number"] += 100
portfolio.update_positions(new_positions)
print(portfolio.positions["Apple"])
print(f"Post-transaction cash: {portfolio.cash:,} USD")

# !! Now the cash position did update properly

Pre-transaction cash: 745,200.0 USD
{'Share price': 150.0, 'Number': 2000}
{'Share price': 150.0, 'Number': 2100}
Post-transaction cash: 262,800.0 USD


In [15]:
portfolio.__dict__.keys()

dict_keys(['_Portfolio__initial_cash', '_Portfolio__positions', 'cash', 'n', 'companies'])

## Advanced methods
Now that we have focused on the attributes of the portfolio, we could also look at more advanced methods for classes like for example class methods and static methods.

Take for example the case where instead of providing a dictionary, we would actually like to construct the positions from a dataframe. This type of behavior can be implemented using class methods. Class methods basically provide an alternative to using the \_\_init\_\_ method.

In [16]:
import pandas as pd
portfolio_df = pd.DataFrame(
    {"company": ["Apple", "Verizon", "Solvay"],
     "share_price": [150, 52.4, 106.3],
     "number": [1000, 2000, 2000]}
)
portfolio_df

,company,share_price,number
0,Apple,150.0,1000
1,Verizon,52.4,2000
2,Solvay,106.3,2000


In [17]:
port = portfolio.from_dataframe(initial_cash, portfolio=portfolio_df)
print(port)

A portfolio consisting of 3 positions and a total value of 1,467,400.0 USD.


In [18]:
port.positions

{'Apple': {'Share price': 150.0, 'Number': 1000},
 'Verizon': {'Share price': 52.4, 'Number': 2000},
 'Solvay': {'Share price': 106.3, 'Number': 2000}}

Sometimes, in a class, you also use methods that do not strictly require a class instance to be executed. For example, the calculation of a present value can be very relevant for certain portfolios e.g. fixed income portfolios, however, the calculation of this value does not require the portfolio to be instantiated. In that case, we define a static method.

In [19]:
print("Present value: {:,}".format(portfolio.get_present_value(fv=110e6, r=0.10, time=1)))

Present value: 100,000,000.0


Instead of using a class static method we could also define the function outside of the class, however, the disadvantage is that now your logic is not contained inside the class. Furthermore, we are now mixing namespaces: https://www.programiz.com/python-programming/namespace.